# Trabalho Prático 1 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## Exercício 1

In [4]:
'''
Imports necessários para a execução deste mesmo Notebook
'''
import io, os
import multiprocessing as mp

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import padding

from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.asymmetric.padding import PSS, MGF1, PKCS1v15

from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat, PrivateFormat, NoEncryption
from cryptography.hazmat.primitives.serialization import load_pem_public_key
from cryptography.hazmat.primitives.serialization import load_pem_private_key

O principal objetivo deste primeiro exercício prático passava pela implementação de uma **comunicação síncrona** e **segura** entre duas entidades, um **receiver** e um **emitter** seguindo determinados parâmetros que vão sendo explicitados e demonstrados em cada uma das alíneas que se seguem.

### Alínea a - Implementar um gerador de nounces

Como se encontra explícito no próprio título desta subsecção, a principal finalidade desta alínea era a implementação de um **gerador de nounces** de modo a que, sempre que seja solicitado uma nova instância de **nounce**, temos a garantia de que ele é **aleatório** e ainda que **nunca foi retornado**, numa ocasião prévia por esse mesmo gerador.

Com isto em mente, desenvolvemos a seguinte classe em **Python**, designada de **NGenerator**, a qual apenas apresenta um método que permite o acesso a um novo **nounce**, designado de **get()**. De modo a implementar essa funcionalidade no **NGenerator**, recorremos a uma lista que vai armazenando os **nounces** que vão sendo retornados, à qual acedemos sempre aquando da solicitação da geração de um novo nounce de modo a verificar se tal nounce já foi retornado previamente. Em caso afirmativo, essa instância recém-gerada **é descartada** e é gerada **uma nova instância** para ser retornada, adicionando-a, previamente ao *historic* do respetivo **NGenerator**.

In [11]:
''' 
Classe que permite gerar 
nounces que nunca se repetem
'''
class NGenerator :

    # Construtor para objetos da classe NGenerator
    def __init__(self,size) :

        self.size = size
        self.historic = []

    ''' 
    Método que nos permite obter um novo nounce
    '''
    def get(self) :

        nounce = os.urandom(self.size)
        while nounce in self.historic :
            nounce = os.urandom(self.size)
        self.historic.append(nounce)
        return nounce

    '''
    Método que nos permnite adicionar um novo nounce 
    ao historico. Método importante para registar os 
    nounces usados pelo peer.
    '''
    def addToHistoric(self,nounce) :

        self.historic.append(nounce)

> De seguida, mostramos o exemplo de execução do método que permite obter uma nova instância de **nounce**.

In [12]:
ng = NGenerator(32)
ng.get()

b'\xfb\xa0\x98X\x18\xd9\x17\xd4\x04\xe6\xd5P\xba~\xf1\r\x82\xd2e\x0fv\xa7\x05\xbf\x8e\x0e\x05D"\xb7\xcb\xe3'

### Implementação da comunicação entre as duas entidades (*Emitter* e *Receiver*)

As alíneas **b** e **c** não podem ser isoladas do mesmo modo que aconteceu com a alínea a, visto que estas últimas representam alguns **requisitos** aos quais a comunicação entre as duas diferentes partes devem obedecer, pelo que falaremos de cada uma delas ao longo da explicação acerca da implementação realizada pelo nosso grupo quando tal seja oportuno.

Numa primeira fase, pensamos que seria necessário ambas as entidades conhecerem a **chave pública** uma da outra, de modo a que fosse possível, para qualquer uma destas, verificarem as **assinaturas** uma da outra, na fase de **handshake**, a qual segue o **algorítmo de Diffie-Hellman**, de modo a ambas as partes acordarem num **chave simétrica** a ser usada para a **cifragem de mensagens** durante a comunicação. Definimos, para além disso, os parâmetros **p** e **g** a serem usados no **Diffie-Hellman**.

In [23]:
# Chaves públicas DSA de emitter e receiver
sender_public_key = None
receiver_public_key = None

# RFC 3526's parameters for Diffie-Hellman. Easier to hardcode...
p = 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AACAA68FFFFFFFFFFFFFFFF
g = 2
params_numbers = dh.DHParameterNumbers(p,g)
parameters = params_numbers.parameters()

Os métodos apresentados já de seguida são responsáveis pela geração de **chaves DSA privadas** e **públicas**, de modo a que seja possível, aquando da fase de **handshake**, para ambos os intervinientes na comunicação se conseguirem autenticar. No nosso contexto de teste em particular, o processo "principal" (método **main**) é responsável por gerar ambas as chaves privadas numa primeira instância, e, de seguida, transmití-las aos respetivos portadores das mesmas, cada um a correr num **processo distinto** (recorremos ao **multiprocessing do Python**).

De seguida, a partir do par de **chaves privadas** gerado, gera, para cada uma, a sua respetiva **chave pública**, esta sim, conhecida de ambas as entidades. 

In [22]:
# Size of dsa keys
DSA_KEY_SIZE = 2048 #bits

''' 
Função que permite gerar uma chave privada e 
que irá ser usada por ambas as entidades, sender e 
emitter para poder realizar a troca de chaves
'''
def generatePrivateKeys() :

    sender_private_key = dsa.generate_private_key(
        key_size=DSA_KEY_SIZE,
    )
    receiver_private_key = dsa.generate_private_key(
        key_size=DSA_KEY_SIZE,
    )
    return sender_private_key,receiver_private_key

'''
Função que recebe o par de chaves privadas geradas 
e gera cada uma das correspondentes chaves públicas
'''
def generatePublicKeys(private_pair) :

    global sender_public_key, receiver_public_key
    sender_public_key = private_pair[0].public_key()
    receiver_public_key = private_pair[1].public_key()

#### Fase de Handshake

Passamos agora para a explicação acerca da fase de handshake, a qual é, obviamente, distinta para os dois intervenientes na comunicação. Começamos então por explicar o procedimento seguido pelo agente **emitter**.

In [7]:
''' 
Função que representa a execução da entidade sender
'''
def sender(conn,stdin):
    message = stdin.recv()

    while len(message) > 0 :
        conn.send(message.encode('utf-8'))
        assert 'ok' == conn.recv()     # para garantir o sincronismo
        ''' Pedimos à thread principal a 
        próxima mensagem lida a partir do stdin'''
        stdin.send('next')
        message = stdin.recv()

    # Para terminar a conexão
    conn.send(message.encode('utf-8'))
    conn.close()
    print('[SENDER] SHUTDOWN')

''' 
Função que representa a execução da entidade receiver 
'''
def receiver(conn):
    try:
        message = bytes(conn.recv()).decode('utf-8')
    except EOFError:
        print('EOFError')
    while len(message) > 0:
        # Imprimimos o conteudo da mensagem recebida
        print('[Receiver] RECEIVED: ' + message)
        conn.send('ok')        # para garantir o sincronismo
        try:
            message = bytes(conn.recv()).decode('utf-8')
        except EOFError:
            break
    conn.close()
    print('[RECEIVER] SHUTDOWN')


''' 
Função responsável por arrancar a 
execução de ambas as entidades, o emitter 
e o receiver. Para além disto é responsável por 
mediar a comunicação entre o stdin e o sender
'''
def main() :
    try:
        mp.set_start_method('fork')
    except:
        print("O start_method já foi inicializado anteriormente ")

    receiver_conn, sender_conn = mp.Pipe()
    receiver_stdin, main_stdin = mp.Pipe()
    s = mp.Process(target=sender, args=(sender_conn,receiver_stdin))
    r = mp.Process(target=receiver, args=(receiver_conn,))
    s.start()
    r.start()

    message = input('[Emitter] > Mensagem a ser enviada: \n')
    while len(message) != 0 and message != 'exit':
        main_stdin.send(message)
        ''' Esperamos a confirmação do sender para 
        receber nova mensagem do stdin'''
        assert 'next' == main_stdin.recv()
        message = input('[Emitter] > Mensagem a ser enviada: \n')
    # Para terminar a ligação
    main_stdin.send('')

In [8]:
main()

O start_method já foi inicializado anteriormente 


[Emitter] > Mensagem a ser enviada: 
 olá tudo bem oh receiver?


[Receiver] RECEIVED: olá tudo bem oh receiver?


[Emitter] > Mensagem a ser enviada: 
 Não falas não é?


[Receiver] RECEIVED: Não falas não é?


[Emitter] > Mensagem a ser enviada: 
 Vou ali e já venho...


[Receiver] RECEIVED: Vou ali e já venho...


[Emitter] > Mensagem a ser enviada: 
 exit


[RECEIVER] SHUTDOWN[SENDER] SHUTDOWN

